# Stage 3: Blocking

### Import Required Packages

In [1]:
from urllib2 import urlopen
import pandas as pd
import re
import numpy as np
import nltk
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.lancaster import LancasterStemmer

### Load & Prepare Tables
1. __Load table data from Github__
    1. Spoj Data = *table_A*
    2. Codechef Data = *table_B*
2. __Combine Text Attributes__
    1. Concatenate _description_ + _input_ + _output_ text attributes into single _words_ attribute
    2. Drop _description_ + _input_ + _output_ text attributes

In [2]:
table_spoj = "https://raw.githubusercontent.com/KaranTalreja/CS638/master/spoj/json/spoj_blocking.csv?token=AKuoLTvhQla5FVgd-sWedBmZ7C9MitFsks5YLxDGwA%3D%3D"
table_A = pd.read_csv(urlopen(table_spoj)).drop("Unnamed: 0", axis=1)
table_A["words"] = table_A["description"].fillna("") + table_A["input"].fillna("") + table_A["output"].fillna("")
table_A = table_A.drop(["description", "input", "output"], axis=1)

In [3]:
table_codechef = "https://raw.githubusercontent.com/KaranTalreja/CS638/master/codechef/data/codechef_blocking.csv?token=AD_cZbkPhGiqHVai0GuHnKYh2i7k3xIlks5YMLsuwA%3D%3D"
table_B = pd.read_csv(urlopen(table_codechef))
table_B["words"] = table_B["description"].fillna("") + table_B["input"].fillna("") + table_B["output"].fillna("")
table_B = table_B.drop(["description", "input", "output"], axis=1)

### Standardize _Words_ Attribute
1. __Create function to standardize words attribute__
    1. Remove numbers and special symbols
    2. Convert all words to lowercase and split by word
    3. Remove common english stopwords and individual alphabetical characters
    4. Rejoin text into single string
2. __Apply *std_words* function to *Table_A* and *Table_B*__

In [4]:
# Step 1
def std_words(df):
    letters_only = re.sub("[^a-zA-Z]", " ", df["words"]) # letters only, drop numbers & symbols
    words = letters_only.lower().split() # lower case, split by word                  
    stops = set((nltk.corpus.stopwords.words("english") + 
                ['a','b','c','d','e','f','g','h','i','j','k','l','m','n',
                'o','p','q','r','s','t','u','v','w','x','y','z'])) # general stopwords & letters  
    st = LancasterStemmer()
    meaningful_words = [st.stem(w) for w in words if not w in stops] # remove stopwords
    return( " ".join( meaningful_words )) # return re-joined string

# Step 2
table_A["words"] = table_A.apply(std_words, axis=1)
table_B["words"] = table_B.apply(std_words, axis=1)

### Create & Apply Bag of Words
1. __Create bag of words from *Table_A*__
    1. Maximum 1000 words
    2. Each word can be in at most 60% of the instances
        1. This is an attempt to prevent using overly common words to block
    3. Simply specify if word occurs in a particular instance, do not track the frequency of occurance
2. __Fit Vectorizer to *Table_A* the apply to both tables__
    1. Must transform the output to an numpy array
3. __Map words to a pandas table for both *Table_A* and *Table_B*__

In [5]:
# Step 1
vectorizer = CountVectorizer(analyzer = "word",
                             max_features = 1000,
                             max_df = 0.6) 

# Step 2
fit_words = vectorizer.fit(table_A["words"])
table_A_features = fit_words.transform(table_A["words"]).toarray()
table_B_features = fit_words.transform(table_B["words"]).toarray()

# Step 3
vocab = fit_words.get_feature_names() # words used from corpus
word_mapping_A = pd.DataFrame(table_A_features, columns=vocab) # which instances contain each vocab word
word_mapping_B = pd.DataFrame(table_B_features, columns=vocab) # which instances contain each vocab word

### Begin Blocking on Text
1. **Map Table_A to inverse index dictionary**
2. **Map Table_B to inverse index dictionary**
3. **Start Blocking on string similarities**
    1. Use basic similarity measure (percentage of overlap, tokenized by word)

In [6]:
# Step 1
inverse_indexing_A = defaultdict(list)
for index, row in word_mapping_A.iterrows():
    words = row[row.values >= 1]
    for word in words.index:
        inverse_indexing_A[word].append(index)

In [7]:
# Step 2
inverse_indexing_B = defaultdict(list)
for index, row in word_mapping_B.iterrows():
    words = row[row.values >= 1]
    for word in words.index:
        inverse_indexing_B[word].append(index)

In [8]:
# Step 3
def blocking_match(row, min_sim=0.5):
    list_of_sim = defaultdict(float)
    words = word_mapping_A.ix[row][word_mapping_A.ix[row].values > 0].index
    for i in words:
        for j in inverse_indexing_B[i]:
            list_of_sim[j] = list_of_sim[j] + (1. / len(words))
    try:
        del list_of_sim[row]
    except:
        pass
    for key in list_of_sim.keys():
        if list_of_sim[key] < min_sim:
            del list_of_sim[key]
    return list_of_sim

# Block Rows
blocked = defaultdict(list)
for row in range(0,len(word_mapping_A)):
    blocked[row] = (blocking_match(row, 0.4).keys())
    if float(row) % round((len(word_mapping_A)/10.)) == 0:
        print "%.0f%% completed." % (100 * float(row) / float((len(word_mapping_A))))
    elif float(row) == len(word_mapping_A)-1:
        print "Finished!"
    else:
        pass

0% completed.
10% completed.
20% completed.
30% completed.
40% completed.
50% completed.
60% completed.
70% completed.
80% completed.
90% completed.
Finished!


### Format as a Table
1. __Columns:__
    1. __Table_A_ID:__ Index from table_A (spoj)
    2. __Table_B_ID:__ Index from table_B (codechef)

In [9]:
compare = pd.DataFrame(columns=["table_A_ID", "table_B_ID"])

count = 0
for key, value in blocked.iteritems():
    if float(count) % round((len(blocked)/10.)) == 0:
        print "%.0f%% completed." % (100 * float(count) / float((len(blocked))))
    elif float(count) == len(blocked)-1:
        print "Finished!"
    else:
        pass
    count = count + 1
    for index in value:
        compare = compare.append([{"table_A_ID": key, "table_B_ID": index}], ignore_index=True)

0% completed.
10% completed.
20% completed.
30% completed.
40% completed.
50% completed.
60% completed.
70% completed.
80% completed.
90% completed.
Finished!


In [10]:
compare

,table_A_ID,table_B_ID
0,0.0,4261.0
1,0.0,4532.0
2,0.0,4767.0
3,0.0,4942.0
4,0.0,5166.0
5,0.0,5362.0
6,1.0,5335.0
7,2.0,5335.0
8,3.0,1582.0
9,3.0,4767.0
